<a href="https://colab.research.google.com/github/vcorredorg/SenalesySistemas2025/blob/main/Taller2/App_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#APP comunicaciones- modulación AM

In [43]:
#instalación de librerías
!pip install streamlit -q


In [44]:
!mkdir pages

mkdir: cannot create directory ‘pages’: File exists


In [45]:
%%writefile modulacion_AM.py

import streamlit as st
st.set_page_config(
    page_title="Comunicaciones AM", page_icon="📡", layout="wide"
)

st.write("# Bienvenido! 👋")

st.sidebar.success("Seleccciona una demo a explorar.")

st.markdown(
    """
    Desde Streamlit se explorarán los conceptos de comunicaciones y modulación AM **👈 Seleccione una opción de la barra lateral** para ver el demo correspondiente.
"""
)

Overwriting modulacion_AM.py


In [46]:

%%writefile 1_📈_Espectro_en_frecuencia.py

import streamlit as st
import time
import numpy as np

st.set_page_config(page_title="Espectro en frecuencia", page_icon="📈")

st.markdown("# Espectro en frecuencia")
st.sidebar.header("Espectro en frecuencia")

st.write("Sea la señal portadora c(t) = Ac cos(2πFct), con Ac, Fc ∈ ℝ, y la señal mensaje m(t) ∈ ℝ.")
st.write("Encuentre el espectro en frecuencia de la señal modulada en amplitud (AM):")

st.latex(r"""
y(t) = \frac{1 + m(t)}{A_c} \, c(t)
""")
st.write("y(t) = (1 + A_c m(t)) c(t) = A_c cos(2πF_c t) + m(t) cos(2πF_c t).")

st.write("Propiedades de Transformada de Fourier que usaremos:")

st.latex(r"""
\mathcal{F}\{\cos(2\pi F_c t)\}
= \tfrac12\big[\delta(f-F_c)+\delta(f+F_c)\big]
""")

st.latex(r"""
\mathcal{F}\{x(t)\cos(2\pi F_c t)\}
= \tfrac12\big[X(f-F_c)+X(f+F_c)\big]
""")

st.write("Denotemos M(f) = 𝓕{m(t)}.")

st.write("Primer término:")

st.latex(r"""
\mathcal{F}\{A_c\cos(2\pi F_c t)\}
= \frac{A_c}{2}\big[\delta(f-F_c)+\delta(f+F_c)\big]
""")

st.write("Segundo término:")

st.latex(r"""
\mathcal{F}\{m(t)\cos(2\pi F_c t)\}
= \tfrac12\big[M(f-F_c)+M(f+F_c)\big]
""")

st.write("Espectro de la AM con portadora:")

st.latex(r"""
Y(f)=\frac{A_c}{2}\big[\delta(f-F_c)+\delta(f+F_c)\big]
+\frac12\big[M(f-F_c)+M(f+F_c)\big]
""")

st.write("Interpretación:")
st.write("""
- Líneas de portadora en ±F_c con amplitud A_c/2.
- Bandas laterales: copias de M(f) desplazadas a ±F_c y escaladas por 1/2.
- Si M(f) está acotado a |f| ≤ B, entonces Y(f) ocupa [F_c−B, F_c+B] y [−F_c−B, −F_c+B].
- Para evitar sobre–modulación, típicamente |m(t)| ≤ A_c.
""")

Writing 1_📈_Espectro_en_frecuencia.py


In [47]:
!mv  1_📈_Espectro_en_frecuencia.py pages/

In [61]:
%%writefile 2_simulación_modulación_AM.py
# pages/2_simulación_modulación_AM.py
import sys, subprocess, importlib
import streamlit as st

st.set_page_config(page_title="AM con audio de YouTube (20–25 s)", page_icon="📻", layout="wide")
st.title("Modulación AM con fragmento de YouTube (20–25 s, μ = 1)")

# ========= FASE A: preparar entorno (sin requirements.txt) =========
NEEDED = ["numpy", "matplotlib", "librosa", "soundfile", "yt-dlp", "imageio-ffmpeg"]

def missing_pkgs(pkgs):
    faltan = []
    for p in pkgs:
        mod = p.replace("-", "_")
        try:
            importlib.import_module(mod)
        except Exception:
            faltan.append(p)
    return faltan

faltan = missing_pkgs(NEEDED)

if faltan:
    st.warning("Faltan dependencias: " + ", ".join(faltan))
    prep = st.button("Preparar entorno (instalar y recargar)")
    if prep:
        with st.status("Instalando dependencias…", expanded=True) as status:
            for p in faltan:
                st.write(f"• pip install {p}")
                subprocess.check_call([sys.executable, "-m", "pip", "install", p])
            status.update(label="Dependencias listas. Recargando…", state="complete")
        st.rerun()
    st.stop()

# ========= IMPORTS ya garantizados =========
import os, glob, tempfile
import numpy as np
import matplotlib.pyplot as plt
import librosa
import soundfile as sf
from yt_dlp import YoutubeDL
from imageio_ffmpeg import get_ffmpeg_exe

# ========= UI de simulación =========
st.subheader("Parámetros de la simulación (μ = 1)")
url = st.text_input("URL de YouTube", value="", help="Descarga 5 s empezando en t = 20 s.")
fs  = st.number_input("Frecuencia de muestreo [Hz]", min_value=8000, max_value=96000, value=44100, step=1000)
Ac  = st.number_input("Amplitud de portadora Ac", min_value=0.1, max_value=10.0, value=1.0, step=0.1)
Fc  = st.number_input("Frecuencia de portadora Fc [Hz]", min_value=500.0, max_value=20000.0, value=8000.0, step=100.0)
mu  = 1.0  # fijo por enunciado

st.caption("Plan B: si YouTube falla, sube un archivo de audio y lo usaremos como mensaje.")
uploaded = st.file_uploader("Subir WAV/MP3/M4A (opcional)", type=["wav","mp3","m4a"])
go = st.button("Descargar y simular AM")

# ========= Utilidades =========
def normalize(x, eps=1e-12):
    return x / (np.max(np.abs(x)) + eps)

def rfft_mag(x, fs):
    X = np.fft.rfft(x)
    f = np.fft.rfftfreq(len(x), d=1.0/fs)
    return f, np.abs(X) / len(x)

def plot_time(sig, fs, title, max_secs=0.03):
    N = len(sig); Tmax = min(N, int(max_secs*fs))
    t = np.arange(Tmax)/fs
    fig = plt.figure()
    plt.plot(t, sig[:Tmax])
    plt.xlabel("Tiempo [s]"); plt.ylabel("Amplitud"); plt.title(title)
    st.pyplot(fig)

def plot_spec(sig, fs, title):
    f, mag = rfft_mag(sig, fs)
    fig = plt.figure()
    plt.plot(f, mag)
    plt.xlabel("Frecuencia [Hz]"); plt.ylabel("|X(f)|"); plt.title(title)
    plt.xlim(0, f[-1])
    st.pyplot(fig)

def to_wav_bytes(x, fs):
    import io
    buf = io.BytesIO()
    sf.write(buf, np.clip(x, -1, 1), fs, format="WAV")
    buf.seek(0)
    return buf

def download_youtube_20_25(url, sr=44100, verbose=False):
    tmpdir = tempfile.mkdtemp()
    outtmpl = os.path.join(tmpdir, "audio.%(ext)s")
    # ffmpeg embebido (evita apt-get)
    ffmpeg_bin = get_ffmpeg_exe()
    ff_dir = os.path.dirname(ffmpeg_bin)
    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": outtmpl,
        "quiet": not verbose,
        "noprogress": True,
        "ffmpeg_location": ff_dir,
        "prefer_ffmpeg": True,
    }
    with YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    candidates = glob.glob(os.path.join(tmpdir, "audio.*"))
    if not candidates:
        raise RuntimeError("No se encontró el audio descargado.")
    path = candidates[0]
    # Carga y recorte exacto por índices
    y, _ = librosa.load(path, sr=sr, mono=True)
    i0, i1 = int(20*sr), int(25*sr)
    if len(y) <= i0:
        seg = y[:int(5*sr)]
    else:
        seg = y[i0:i1]
        if len(seg) < int(5*sr):
            seg = np.pad(seg, (0, int(5*sr)-len(seg)))
    return seg, sr

# ========= Pipeline principal =========
if go:
    with st.status("Ejecutando simulación…", expanded=True) as status:
        # 1) Obtener m(t)
        if uploaded is not None:
            st.write("• Usando archivo subido como mensaje")
            y, sr_in = librosa.load(uploaded, sr=int(fs), mono=True)
            m = y[:int(5*fs)]
            fs_used = int(fs)
        else:
            if not url:
                st.error("Debes ingresar una URL de YouTube o subir un archivo.")
                st.stop()
            try:
                st.write("• Descargando fragmento 20–25 s de YouTube…")
                m, fs_used = download_youtube_20_25(url, sr=int(fs), verbose=True)
            except Exception as e:
                st.error(f"No se pudo descargar/leer audio de YouTube: {e}")
                st.info("Sube un archivo de audio como alternativa.")
                st.stop()

        # 2) Normalizar (μ=1)
        st.write("• Normalizando mensaje para μ=1 (sin sobre-modulación)")
        m = normalize(m)
        N = len(m); t = np.arange(N)/fs_used

        # 3) Portadora y AM
        st.write("• Generando portadora y señal AM")
        c = Ac * np.cos(2*np.pi*Fc*t)
        y = Ac * (1.0 + mu*m) * np.cos(2*np.pi*Fc*t)

        status.update(label="Simulación completada", state="complete")

    # ======= Reproducción =======
    st.subheader("Reproducción (opcional)")
    st.caption("Baja el volumen: la portadora y la AM pueden sonar agudas.")
    st.write("Mensaje m(t)"); st.audio(to_wav_bytes(m, fs_used).getvalue(), format="audio/wav")
    st.write("Portadora c(t)"); st.audio(to_wav_bytes(c/(np.max(np.abs(c))+1e-12), fs_used).getvalue(), format="audio/wav")
    st.write("Señal AM y(t)"); st.audio(to_wav_bytes(y/(np.max(np.abs(y))+1e-12), fs_used).getvalue(), format="audio/wav")

    # ======= Gráficas =======
    st.subheader("Gráficas en el tiempo (ventana corta ~30 ms)")
    col1, col2, col3 = st.columns(3)
    with col1: plot_time(m, fs_used, "m(t)")
    with col2: plot_time(c, fs_used, "c(t)")
    with col3: plot_time(y, fs_used, "y(t)")

    st.subheader("Espectro (magnitud, rFFT)")
    col4, col5, col6 = st.columns(3)
    with col4: plot_spec(m, fs_used, "M(f)")
    with col5: plot_spec(c, fs_used, "C(f)")
    with col6: plot_spec(y, fs_used, "Y(f)")
else:
    st.info("Paso 1: si la app te avisó de dependencias faltantes, pulsa **Preparar entorno**. Paso 2: ingresa URL o sube audio y pulsa **Descargar y simular AM**.")


Writing 2_simulación_modulación_AM.py


In [62]:
!mv 2_simulación_modulación_AM.py pages/

In [50]:

%%writefile 3_📊_DataFrame_Demo.py

import streamlit as st
import pandas as pd
import altair as alt
from urllib.error import URLError

st.set_page_config(page_title="DataFrame Demo", page_icon="📊")

st.markdown("# DataFrame Demo")
st.sidebar.header("DataFrame Demo")
st.write(
    """Esta demo muestra como usar `st.write` para visualizar Pandas DataFrames.
(Datos cortesia de [UN Data Explorer](http://data.un.org/Explorer.aspx).)"""
)


@st.cache_data
def get_UN_data():
    AWS_BUCKET_URL = "http://streamlit-demo-data.s3-us-west-2.amazonaws.com"
    df = pd.read_csv(AWS_BUCKET_URL + "/agri.csv.gz")
    return df.set_index("Region")


try:
    df = get_UN_data()
    countries = st.multiselect(
        "Seleccione los paises", list(df.index), ["China", "United States of America"]
    )
    if not countries:
        st.error("Por favor seleccione al menos un pais.")
    else:
        data = df.loc[countries]
        data /= 1000000.0
        st.write("### Producción Agricola Bruta ($B)", data.sort_index())

        data = data.T.reset_index()
        data = pd.melt(data, id_vars=["index"]).rename(
            columns={"index": "year", "value": "Gross Agricultural Product ($B)"}
        )
        chart = (
            alt.Chart(data)
            .mark_area(opacity=0.3)
            .encode(
                x="year:T",
                y=alt.Y("Gross Agricultural Product ($B):Q", stack=None),
                color="Region:N",
            )
        )
        st.altair_chart(chart, use_container_width=True)
except URLError as e:
    st.error(
        """
        **Esta demo requiere conexión a internet.**
        Connection error: %s
    """
        % e.reason
    )

Writing 3_📊_DataFrame_Demo.py


In [51]:

!mv 3_📊_DataFrame_Demo.py pages/

In [63]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

#Ejecutar Streamlit
!streamlit run modulacion_AM.py &>/content/logs.txt &

#Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

#Leer la URL pública generada por Cloudflare
import time
time.sleep(5)  # Esperar que se genere la URL

import re
found_context = False  # Indicador para saber si estamos en la sección correcta

with open('/content/cloudflared.log') as f:
    for line in f:
        #Detecta el inicio del contexto que nos interesa
        if "Your quick Tunnel has been created" in line:
            found_context = True

        #Busca una URL si ya se encontró el contexto relevante
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)  #Extrae la URL encontrada
                print(f'Tu aplicación está disponible en: {url}')
                break  #Termina el bucle después de encontrar la URL

--2025-11-10 14:58:31--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64 [following]
--2025-11-10 14:58:31--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/955e9d1b-ac5e-4188-8867-e5f53958a8fe?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-10T15%3A56%3A49Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-10

In [53]:
import os

res = input("Digite (1) para finalizar la ejecución del Dashboard: ")

if res.upper() == "1":
    os.system("pkill streamlit")  # Termina el proceso de Streamlit
    print("El proceso de Streamlit ha sido finalizado.")


Digite (1) para finalizar la ejecución del Dashboard: 0
